# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [99]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [100]:
stocks = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
stocks = stocks[['Symbol', 'Security']]
stocks = stocks.rename(columns={'Symbol': 'Ticker', 'Security': 'Name'})
stocks[stocks['Ticker'] == 'BRK.B'] = ['BRK-B', 'Berkshire Hathaway']
stocks[stocks['Ticker'] == 'BF.B'] = ['BF-B', 'Brown-Forman']
stocks

,Ticker,Name
0,MMM,3M
1,AOS,A. O. Smith
2,ABT,Abbott
3,ABBV,AbbVie
4,ACN,Accenture
...,...,...
498,XYL,Xylem Inc.
499,YUM,Yum! Brands
500,ZBRA,Zebra Technologies
501,ZBH,Zimmer Biomet


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [101]:
stock = yf.Ticker('BRK-A')
info = stock.info
info

{'address1': '3555 Farnam Street',
 'city': 'Omaha',
 'state': 'NE',
 'zip': '68131',
 'country': 'United States',
 'phone': '402 346 1400',
 'website': 'https://www.berkshirehathaway.com',
 'industry': 'Insurance - Diversified',
 'industryKey': 'insurance-diversified',
 'industryDisp': 'Insurance - Diversified',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': 'Berkshire Hathaway Inc., through its subsidiaries, engages in the insurance, freight rail transportation, and utility businesses worldwide. The company provides property, casualty, life, accident, and health insurance and reinsurance; and operates railroad systems in North America. It also generates, transmits, stores, and distributes electricity from natural gas, coal, wind, solar, hydroelectric, nuclear, and geothermal sources; operates natural gas distribution and storage facilities, interstate pipelines, liquefied natural gas facilities, and co

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [102]:
trailing_pe = info.get('trailingPE', None)
forward_pe = info.get('forwardPE', None)
price = info.get('previousClose', None)
print(forward_pe, price)

# I want forward

21.672571 620040.0


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [103]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [104]:
final_dataframe = pd.DataFrame(columns=my_columns)
def fetch_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        if info.get('forwardPE', None) is not None:
            price = info.get('previousClose', None)
            pe_forward = info.get('forwardPE', None)
        else:
            print(f"No data for {ticker}")
            price = None
            pe_forward = None
        
        return pd.Series(
            [
                ticker,
                price,
                pe_forward,
                'N/A'
            ],
            index=my_columns
        )
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.Series([ticker, None, None, 'N/A'], index=my_columns)

# Using ThreadPoolExecutor to fetch data concurrently
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_stock_data, ticker) for ticker in stocks['Ticker']]
    
    for future in as_completed(futures):
        final_dataframe = pd.concat([final_dataframe, future.result().to_frame().T], ignore_index=True)

final_dataframe

No data for NWS


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,124.61,17.369959,N/A
1,APD,278.47,20.8503,N/A
2,AES,16.29,7.904307,N/A
3,ADBE,509.32,27.264051,N/A
4,ABT,108.63,23.248407,N/A
...,...,...,...,...
498,WTW,274.34,14.655983,N/A
499,ZBRA,317.86,20.717278,N/A
500,WYNN,73.55,12.996593,N/A
501,YUM,133.32,21.517296,N/A


In [105]:
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'].notnull()]
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,124.61,17.369959,N/A
1,APD,278.47,20.8503,N/A
2,AES,16.29,7.904307,N/A
3,ADBE,509.32,27.264051,N/A
4,ABT,108.63,23.248407,N/A
...,...,...,...,...
498,WTW,274.34,14.655983,N/A
499,ZBRA,317.86,20.717278,N/A
500,WYNN,73.55,12.996593,N/A
501,YUM,133.32,21.517296,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [106]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe

C:\Users\jsullij9\AppData\Local\Temp\ipykernel_21944\1585619539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,UAL,37.88,3.397548,N/A
1,GM,39.95,4.048853,N/A
2,VTRS,11.56,4.172794,N/A
3,AAL,9.28,5.115385,N/A
4,DAL,37.93,5.214189,N/A
5,EG,356.3,5.23715,N/A
6,F,9.71,5.556818,N/A
7,APA,27.59,5.636175,N/A
8,WBA,10.72,5.865591,N/A
9,WDC,56.99,6.483616,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [107]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        print("$", "{:,}".format(val))
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [108]:
portfolio_input()

$ 1,000,000.0


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [109]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,UAL,37.88,3.397548,527
1,GM,39.95,4.048853,500
2,VTRS,11.56,4.172794,1730
3,AAL,9.28,5.115385,2155
4,DAL,37.93,5.214189,527
5,EG,356.3,5.23715,56
6,F,9.71,5.556818,2059
7,APA,27.59,5.636175,724
8,WBA,10.72,5.865591,1865
9,WDC,56.99,6.483616,350


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [110]:
rv_columns = ['Ticker',
              'Price', 
              'Number of Shares to Buy', 
              'P/E Ratio', 
              'P/E Percentile',  
              'P/B Ratio', 
              'P/B Percentile', 
              'P/S Ratio', 
              'P/S Percentile', 
              'EV/EBITDA', 
              'EV/EBITDA Percentile', 
              'EV/GP', 
              'EV/GP Percentile', 
              'RV Score']

rv_dataframe = pd.DataFrame(columns=rv_columns)

def get_financial_ratios(ticker):
    stock = yf.Ticker(ticker)
    
    # Downloading the necessary data
    try:
        info = stock.info
        financials = stock.financials
        
        price = info.get('previousClose', None)
        pe_ratio = info.get('forwardPE', None)
        pb_ratio = info.get('priceToBook', None)
        ps_ratio = info.get('priceToSalesTrailing12Months', None)
        
        enterprise_value = info.get('enterpriseValue', None)
        ebitda = financials.loc['EBITDA'].iloc[0] if 'EBITDA' in financials.index else None
        gross_profit = financials.loc['Gross Profit'].iloc[0] if 'Gross Profit' in financials.index else None
        
        ev_ebitda = enterprise_value / ebitda if ebitda else None
        ev_gp = enterprise_value / gross_profit if gross_profit else None
        
        return pd.Series(
            [
                ticker,
                price,
                'N/A',  # Number of shares to buy
                pe_ratio,
                'N/A',  # P/E percentile
                pb_ratio,
                'N/A',  # P/B percentile
                ps_ratio,
                'N/A',  # P/S percentile
                ev_ebitda,
                'N/A',  # EV/EBITDA percentile
                ev_gp,
                'N/A',  # EV/GP percentile
                'N/A'  # RV score
            ],
            index=rv_columns
        )
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")
        return pd.Series([ticker] + ['N/A'] * (len(rv_columns) - 1), index=rv_columns)

get_financial_ratios('MMM')

Ticker                           MMM
Price                         124.61
Number of Shares to Buy          N/A
P/E Ratio                  17.369959
P/E Percentile                   N/A
P/B Ratio                  17.521044
P/B Percentile                   N/A
P/S Ratio                   2.103661
P/S Percentile                   N/A
EV/EBITDA                 -10.649498
EV/EBITDA Percentile             N/A
EV/GP                       5.067584
EV/GP Percentile                 N/A
RV Score                         N/A
dtype: object

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [111]:
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(get_financial_ratios, ticker) for ticker in stocks['Ticker']]
    
    for future in as_completed(futures):
        rv_dataframe = pd.concat([rv_dataframe, future.result().to_frame().T], ignore_index=True)

rv_dataframe        

Error processing ticker FDX: unsupported operand type(s) for /: 'NoneType' and 'float'
Error processing ticker INTU: unsupported operand type(s) for /: 'NoneType' and 'float'
Error processing ticker WDC: unsupported operand type(s) for /: 'NoneType' and 'float'


,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,124.61,N/A,17.369959,N/A,17.521044,N/A,2.103661,N/A,-10.649498,N/A,5.067584,N/A,N/A
1,ACN,313.74,N/A,24.955362,N/A,7.197578,N/A,3.095982,N/A,18.555322,N/A,9.476185,N/A,N/A
2,ABT,108.63,N/A,23.248407,N/A,4.845561,N/A,4.678184,N/A,18.671576,N/A,8.894601,N/A,N/A
3,AOS,81.65,N/A,18.815668,N/A,6.230259,N/A,3.031461,N/A,14.372509,N/A,7.974187,N/A,N/A
4,AES,16.29,N/A,7.904307,N/A,3.799448,N/A,0.944924,N/A,17.127776,N/A,17.449264,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,WELL,111.04,N/A,66.06896,N/A,2.412744,N/A,9.777807,N/A,33.285655,N/A,31.584249,N/A,N/A
499,XYL,126.14,N/A,26.650627,N/A,2.992272,N/A,3.685343,N/A,28.512955,N/A,11.753592,N/A,N/A
500,ZTS,174.82,N/A,28.99687,N/A,16.739542,N/A,9.680203,N/A,23.073791,N/A,14.138145,N/A,N/A
501,ZBH,107.04,N/A,12.623271,N/A,1.789044,N/A,3.024746,N/A,12.44936,N/A,5.204894,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [112]:
rv_dataframe[rv_dataframe.isna().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
9,AFL,97.17,N/A,14.085714,N/A,2.125046,N/A,2.860473,N/A,None,N/A,None,N/A,N/A
20,MO,49.35,N/A,10.160165,N/A,None,N/A,4.179808,N/A,8.703229,N/A,7.524235,N/A,N/A
21,ALL,169.02,N/A,9.396236,N/A,2.701598,N/A,0.74345,N/A,None,N/A,None,N/A,N/A
25,AAL,9.28,N/A,5.115385,N/A,None,N/A,0.11439,N/A,6.62214,N/A,3.095191,N/A,N/A
28,AXP,225.92,N/A,16.919474,N/A,5.570633,N/A,2.827645,N/A,None,N/A,None,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,VRSN,178.62,N/A,20.43368,N/A,None,N/A,11.315238,N/A,16.923361,N/A,14.312634,N/A,N/A
480,WRB,54.73,N/A,12.784223,N/A,2.697938,N/A,1.635237,N/A,None,N/A,None,N/A,N/A
489,WFC,52.12,N/A,9.502722,N/A,1.114778,N/A,2.278055,N/A,None,N/A,None,N/A,N/A
493,WYNN,73.55,N/A,12.996593,N/A,None,N/A,1.226473,N/A,9.824597,N/A,5.999714,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [113]:
for column in ['P/E Ratio', 'P/B Ratio', 'P/S Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column] = pd.to_numeric(rv_dataframe[column], errors='coerce')
    rv_dataframe[column] = rv_dataframe[column].fillna(rv_dataframe[column].mean())



Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [114]:
rv_dataframe[rv_dataframe.isna().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [115]:
metrics = {
    'P/E Ratio': 'P/E Percentile', 
    'P/B Ratio':'P/B Percentile',
     'P/S Ratio': 'P/S Percentile', 
     'EV/EBITDA': 'EV/EBITDA Percentile', 
     'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,124.61,N/A,17.369959,49.304175,17.521044,93.240557,2.103661,35.785288,-10.649498,1.39165,5.067584,13.32008,N/A
1,ACN,313.74,N/A,24.955362,76.341948,7.197578,70.775348,3.095982,53.677932,18.555322,67.395626,9.476185,43.33996,N/A
2,ABT,108.63,N/A,23.248407,72.166998,4.845561,58.250497,4.678184,69.383698,18.671576,68.588469,8.894601,39.761431,N/A
3,AOS,81.65,N/A,18.815668,56.461233,6.230259,66.003976,3.031461,51.888668,14.372509,35.984095,7.974187,34.592445,N/A
4,AES,16.29,N/A,7.904307,4.373757,3.799448,48.906561,0.944924,11.928429,17.127776,49.900596,17.449264,87.872763,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,WELL,111.04,N/A,66.068960,98.60835,2.412744,32.604374,9.777807,91.451292,33.285655,93.439364,31.584249,98.409543,N/A
499,XYL,126.14,N/A,26.650627,80.914513,2.992272,42.147117,3.685343,60.437376,28.512955,90.258449,11.753592,68.389662,N/A
500,ZTS,174.82,N/A,28.996870,84.691849,16.739542,92.445328,9.680203,91.053678,23.073791,81.312127,14.138145,79.125249,N/A
501,ZBH,107.04,N/A,12.623271,26.640159,1.789044,19.284294,3.024746,51.491054,12.449360,28.031809,5.204894,14.512922,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [116]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.values():
        value_percentiles.append(rv_dataframe.loc[row, metric])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,124.61,N/A,17.369959,49.304175,17.521044,93.240557,2.103661,35.785288,-10.649498,1.39165,5.067584,13.32008,38.60835
1,ACN,313.74,N/A,24.955362,76.341948,7.197578,70.775348,3.095982,53.677932,18.555322,67.395626,9.476185,43.33996,62.306163
2,ABT,108.63,N/A,23.248407,72.166998,4.845561,58.250497,4.678184,69.383698,18.671576,68.588469,8.894601,39.761431,61.630219
3,AOS,81.65,N/A,18.815668,56.461233,6.230259,66.003976,3.031461,51.888668,14.372509,35.984095,7.974187,34.592445,48.986083
4,AES,16.29,N/A,7.904307,4.373757,3.799448,48.906561,0.944924,11.928429,17.127776,49.900596,17.449264,87.872763,40.596421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,WELL,111.04,N/A,66.068960,98.60835,2.412744,32.604374,9.777807,91.451292,33.285655,93.439364,31.584249,98.409543,82.902584
499,XYL,126.14,N/A,26.650627,80.914513,2.992272,42.147117,3.685343,60.437376,28.512955,90.258449,11.753592,68.389662,68.429423
500,ZTS,174.82,N/A,28.996870,84.691849,16.739542,92.445328,9.680203,91.053678,23.073791,81.312127,14.138145,79.125249,85.725646
501,ZBH,107.04,N/A,12.623271,26.640159,1.789044,19.284294,3.024746,51.491054,12.449360,28.031809,5.204894,14.512922,27.992048


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [117]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,WBA,10.72,N/A,5.865591,2.186879,0.690375,1.590457,0.064716,0.198807,-17.039236,0.596421,1.625122,0.795229,1.073559
1,UAL,37.88,N/A,3.397548,0.596421,1.212121,7.157058,0.229330,1.39165,4.017138,2.385686,2.070020,1.192843,2.544732
2,WBD,7.88,N/A,-757.000060,0.198807,0.419902,0.596421,0.457105,4.970179,2.681069,1.789264,3.570555,5.964215,2.703777
3,CVS,57.81,N/A,7.489089,3.777336,0.987475,3.777336,0.203631,0.994036,7.773705,10.536779,2.598852,2.385686,4.294235
4,BWA,32.37,N/A,6.911063,2.584493,1.190983,6.560636,0.505977,5.367793,6.167000,6.361829,3.952835,8.349901,5.84493
5,HPE,17.35,N/A,8.214286,5.5666,1.032687,4.17495,0.791866,9.542744,6.368519,6.95825,3.111176,3.976143,6.043738
6,BG,95.41,N/A,9.876684,11.928429,1.349713,9.145129,0.241858,1.590457,4.750614,3.578529,3.939725,8.151093,6.878728
7,CNC,77.43,N/A,10.236093,14.910537,1.450825,11.729622,0.275459,2.584493,6.810987,8.151093,2.209096,1.988072,7.872763
8,VTRS,11.56,N/A,4.172794,0.994036,0.675073,1.39165,0.879763,11.530815,8.778334,14.115308,4.794367,12.524851,8.111332
9,BEN,21.46,N/A,8.130597,5.367793,0.883868,2.7833,1.380863,22.266402,6.996597,8.548708,2.186939,1.590457,8.111332


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [118]:
portfolio_input()

$ 1,000,000.0


In [119]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe.index)):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[i, 'Price'])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,WBA,10.72,1865,5.865591,2.186879,0.690375,1.590457,0.064716,0.198807,-17.039236,0.596421,1.625122,0.795229,1.073559
1,UAL,37.88,527,3.397548,0.596421,1.212121,7.157058,0.229330,1.39165,4.017138,2.385686,2.070020,1.192843,2.544732
2,WBD,7.88,2538,-757.000060,0.198807,0.419902,0.596421,0.457105,4.970179,2.681069,1.789264,3.570555,5.964215,2.703777
3,CVS,57.81,345,7.489089,3.777336,0.987475,3.777336,0.203631,0.994036,7.773705,10.536779,2.598852,2.385686,4.294235
4,BWA,32.37,617,6.911063,2.584493,1.190983,6.560636,0.505977,5.367793,6.167000,6.361829,3.952835,8.349901,5.84493
5,HPE,17.35,1152,8.214286,5.5666,1.032687,4.17495,0.791866,9.542744,6.368519,6.95825,3.111176,3.976143,6.043738
6,BG,95.41,209,9.876684,11.928429,1.349713,9.145129,0.241858,1.590457,4.750614,3.578529,3.939725,8.151093,6.878728
7,CNC,77.43,258,10.236093,14.910537,1.450825,11.729622,0.275459,2.584493,6.810987,8.151093,2.209096,1.988072,7.872763
8,VTRS,11.56,1730,4.172794,0.994036,0.675073,1.39165,0.879763,11.530815,8.778334,14.115308,4.794367,12.524851,8.111332
9,BEN,21.46,931,8.130597,5.367793,0.883868,2.7833,1.380863,22.266402,6.996597,8.548708,2.186939,1.590457,8.111332


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [120]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [121]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [122]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['P/E Ratio', integer_template],
    'E': ['P/E Percentile', percent_template],
    'F': ['P/B Ratio', integer_template],
    'G': ['P/B Percentile', percent_template],
    'H': ['P/S Ratio', integer_template],
    'I': ['P/S Percentile', percent_template],
    'J': ['EV/EBITDA', integer_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', integer_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)



## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [123]:
writer.close()